Etape préliminaire 

Avant de commencer le projet, il faut installer les packages dont nous allons avoir besoin

In [1]:
import requests
import pandas as pd
import zipfile
import os

Etape 1 : Importation des données

Afin que cette étape se déroule bien, il faut récupérer la base DS_BPE_2024_data.csv préalablement importée sur S3.

In [3]:
# =========================
# ÉTAPE 1 : DONNÉES ÉLECTORALES
# =========================

url_legisl = "https://www.data.gouv.fr/api/1/datasets/r/ab337c6f-e7e8-4981-843c-45052b71096b"
path_legisl = "/home/onyxia/work/Projet-Python-S3/code/results_legisl.xlsx"

# Téléchargement du fichier Excel
response = requests.get(url_legisl)
response.raise_for_status()

with open(path_legisl, "wb") as f:
    f.write(response.content)

# Lecture du fichier Excel
df_legisl = pd.read_excel(path_legisl, sheet_name=0)

print("Le dataframe df_df_legisl peut être traité comme un data frame classique avec pandas")


# =========================
# ÉTAPE 2 : DONNÉES INFRASTRUCTURES (S3 SSPCloud)
# =========================

# Chemin S3 du fichier
path_infra_s3 = "s3://hantier/DS_BPE_2024_data.csv"

# Lecture directe depuis S3
df_infrastruct = pd.read_csv("s3://hantier/DS_BPE_2024_data.csv", sep=';')  # le séparateur utilisé est un ;
print("Le dataframe df_infrastruct peut être traité comme un data frame classique avec pandas")

Le dataframe df_df_legisl peut être traité comme un data frame classique avec pandas
Le dataframe df_infrastruct peut être traité comme un data frame classique avec pandas


/tmp/ipykernel_4467/358226819.py:29: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_infrastruct = pd.read_csv("s3://hantier/DS_BPE_2024_data.csv", sep=';')  # le séparateur utilisé est un ;


Etape 2 : Préparation du dataframe que nous allons utiliser

1) Préparation des données législatives

Résultat recherché : notre but est d'obtenir une table avec des variables permettant d'identifier la commune ('Code département', 'Libellé département', 'Code commune', 'Libellé commune') et une variable correspondant au bord politique de la commune 'bord_pol'.

Ici, la variable nuance candidat prend en valeur un code qui correspond au parti politique du candidat. Nous allons remplacer cette variable par une variable numérique qui attribuera un score allant de - 2 (extrême gauche) à 2 (extrême droite) ce qui nous permettra ensuite en faisant une moyenne pondérée des trois candidats les plus plébiscités afin, ensuite de placer les infrastructures sur un axes gauche-droite du bord politique de la commune

Les variables nuance prennent les valeurs suivantes : DIV, DSV, DVC, DVD, DVG, ECO, ENS, EXD, EXG, HOR, LR, REG, REC, RN, SOC, UDI, UG, UXD, COM, UDI

On crée un dataframe (bord_pol) qui permet de rapidement faire le changement

In [14]:
liste_var_can = ["Nuance candidat "+ str(i) for i in range (1,205)]
liste_var_voix= ["% Voix/exprimés "+ str(i) for i in range (1,205)]
liste_var = ["Code département", "Libellé département", "Code commune", "Libellé commune"] + liste_var_can + liste_var_voix
df_legisl_1 = df_legisl[liste_var]
print("Informations sur le dataframe df_legisl_1")
print(df_legisl_1.columns)
print(df_legisl_1.size)
print(df_legisl_1.head())

bord_pol = [["DIV", "Divers", 1000000], 
["DSV", "Divers souverainiste", 1], 
["DVC", "Divers centre", 0], 
["DVD", "Divers droite", 2], 
["DVG", "Divers gauche", -2], 
["ECO", "Ecologistes", -2], 
["ENS", "Ensemble (EM!)", 1], 
["EXD", 3], 
["EXG", -3], 
["HOR", "Horizon", 1], 
["LR", "Les Républicains", 2], 
["REG", "Régionalistes", 10000000], 
["REC", "Reconquête",  3], 
["RN", "Rassemblement National", 3], 
["SOC", "Socialistes", -2], 
["UDI", "Union des démocrates et indépendants", 1], 
["UG", "Union de la gauche", 2], 
["UXD", "Union de l'extrême droite", 3], 
["COM", "Communistes", 3], 
["UDI", "Union des démocrates et indépendants", -1]]

df_politique = pd.DataFrame(bord_pol, columns=["code", "libelle", "score"])
print (df_politique)

Informations sur le dataframe df_legisl_1
Index(['Code département', 'Libellé département', 'Code commune',
       'Libellé commune', 'Nuance candidat 1', 'Nuance candidat 2',
       'Nuance candidat 3', 'Nuance candidat 4', 'Nuance candidat 5',
       'Nuance candidat 6',
       ...
       '% Voix/exprimés 195', '% Voix/exprimés 196', '% Voix/exprimés 197',
       '% Voix/exprimés 198', '% Voix/exprimés 199', '% Voix/exprimés 200',
       '% Voix/exprimés 201', '% Voix/exprimés 202', '% Voix/exprimés 203',
       '% Voix/exprimés 204'],
      dtype='object', length=412)
14515584
  Code département Libellé département Code commune          Libellé commune  \
0                1                 Ain         1001  L'Abergement-Clémenciat   
1                1                 Ain         1002    L'Abergement-de-Varey   
2                1                 Ain         1004        Ambérieu-en-Bugey   
3                1                 Ain         1005      Ambérieux-en-Dombes   
4            

2) Préparation des données infrastructures
Dans un second temps, nous devons préparer le dataframe des infrastructures. Nous allons le passer d'un format long à un format large, autrement dit, au lieu d'avoir une ligne par infrastructure, nous aurons une ligne par ville et une colonne par infrastructure.

In [5]:
print("Informations sur le dataframe df_infrastruct")
print(df_infrastruct.columns)
print(df_infrastruct.size)
print(df_infrastruct.head())

Informations sur le dataframe df_infrastruct
Index(['GEO', 'GEO_OBJECT', 'FACILITY_DOM', 'FACILITY_SDOM', 'FACILITY_TYPE',
       'BPE_MEASURE', 'TIME_PERIOD', 'OBS_VALUE'],
      dtype='object')
18403840
     GEO GEO_OBJECT FACILITY_DOM FACILITY_SDOM FACILITY_TYPE BPE_MEASURE  \
0  29305     UU2020            D            D1          D115  FACILITIES   
1  77205     UU2020            D            D2          D265  FACILITIES   
2  31113     UU2020            D            D2          D249  FACILITIES   
3  41501     UU2020            D            D1          D112  FACILITIES   
4  46104     UU2020            D            D2          D266  FACILITIES   

   TIME_PERIOD  OBS_VALUE  
0         2024          1  
1         2024          4  
2         2024          1  
3         2024          1  
4         2024          1  


In [6]:
df_infra_large = (
    df_infrastruct
    .query("TIME_PERIOD == 2024")
    .groupby(["GEO", "FACILITY_TYPE"], as_index=False)["OBS_VALUE"]
    .sum()
    .pivot(index="GEO", columns="FACILITY_TYPE", values="OBS_VALUE")
    .fillna(0)
    .reset_index()
)


In [7]:
print("Vérifications sur le dataframe df_infra_large")
print(df_infra_large.columns)
print(df_infra_large.size)
print(df_infra_large.head())

Vérifications sur le dataframe df_infra_large
Index(['GEO', 'A101', 'A104', 'A105', 'A108', 'A109', 'A120', 'A121', 'A122',
       'A124',
       ...
       'F312', 'F313', 'F314', 'F315', 'G101', 'G102', 'G103', 'G104', 'G105',
       '_T'],
      dtype='object', name='FACILITY_TYPE', length=231)
15345330
FACILITY_TYPE GEO  A101  A104  A105  A108  A109  A120  A121  A122  A124  ...  \
0               1   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
1               2   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
2               3   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
3               4   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
4               6   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   

FACILITY_TYPE  F312  F313  F314  F315  G101  G102  G103  G104  G105       _T  
0               0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  56142.0  
1               0.0   0.0   0.0   0.0   0.0   0.0   0

Une fois que nous avons réalisé cette étape, nous allons pouvoir joindre des bases.